### Import the necessary database

In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [2]:
#In[2]:
# define function
import src.SAT_function as data_process
import src.Data_Preprocess as preprosess

In [3]:
# import src.slurm_cluster as scluster
# client, scluster = scluster.init_dask_slurm_cluster(scale=4, cores=40, memory="200GB")

In [4]:
def func_mk(x):
    """
    Mann-Kendall test for trend
    """
    results = data_process.apply_mannkendall(x)
    slope = results[0]
    p_val = results[1]
    return slope, p_val

In [5]:
# Input the MMEM of SAT-OBS internal variability
dir_residuals = '/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/Figure4/data/'
ds_MME_HadCRUT5_1850_2022 = xr.open_mfdataset(dir_residuals + '1850_2022_Internal_SOP_mean.nc')

In [8]:
ds_MME_HadCRUT5_1850_2022

<xarray.Dataset>
Dimensions:  (year: 173)
Coordinates:
  * year     (year) int64 1850 1851 1852 1853 1854 ... 2018 2019 2020 2021 2022
    height   float64 ...
Data variables:
    tas      (year) float64 dask.array<chunksize=(173,), meta=np.ndarray>

In [9]:
# Generate the running windows of the residuals of SAT-OBS
#       with a series of equal length with an interval of 5 years starting from 10 years to 100 years
#       and calculate the trend pattern of each segment
#       and calculate the ensemble standard deviation of the trend pattern of each interval of segments

# define the function to generate the running windows of the residuals of SAT-OBS
def generate_segments(data, segment_length):
    """
    data: 3D array with dimensions [year, lat, lon]
    segment_length: length of each segment in years
    """
    years = range(int(data['year'].min().item()), int(data['year'].max().item()) - segment_length + 2)
    print(years)
    # Initialize an empty list to store the segments
    segments = []
    
    # For each year in the range
    for year in years:
        # Extract the segment of data from that year to year + segment_length
        segment = data.sel(year=slice(str(year), str(year + segment_length - 1)))
        
        # Append this segment to the list of segments
        segments.append(segment)
    
    return segments

In [10]:
time_interval = np.arange(10, 74, 1)

In [11]:
time_interval

array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
       27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43,
       44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60,
       61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73])

In [12]:
# Generate the running windows of the residuals of SAT-OBS
ICV_segments = {}
for i in time_interval:
    ICV_segments[i] = generate_segments(ds_MME_HadCRUT5_1850_2022['tas'], segment_length=i)

range(1850, 2014)
range(1850, 2013)
range(1850, 2012)
range(1850, 2011)
range(1850, 2010)
range(1850, 2009)
range(1850, 2008)
range(1850, 2007)
range(1850, 2006)
range(1850, 2005)
range(1850, 2004)
range(1850, 2003)
range(1850, 2002)
range(1850, 2001)
range(1850, 2000)
range(1850, 1999)
range(1850, 1998)
range(1850, 1997)
range(1850, 1996)
range(1850, 1995)
range(1850, 1994)
range(1850, 1993)
range(1850, 1992)
range(1850, 1991)
range(1850, 1990)
range(1850, 1989)
range(1850, 1988)
range(1850, 1987)
range(1850, 1986)
range(1850, 1985)
range(1850, 1984)
range(1850, 1983)
range(1850, 1982)
range(1850, 1981)
range(1850, 1980)
range(1850, 1979)
range(1850, 1978)
range(1850, 1977)
range(1850, 1976)
range(1850, 1975)
range(1850, 1974)
range(1850, 1973)
range(1850, 1972)
range(1850, 1971)
range(1850, 1970)
range(1850, 1969)
range(1850, 1968)
range(1850, 1967)
range(1850, 1966)
range(1850, 1965)
range(1850, 1964)
range(1850, 1963)
range(1850, 1962)
range(1850, 1961)
range(1850, 1960)
range(1850

In [13]:
type(ICV_segments)

dict

In [14]:
ICV_segments.keys()

dict_keys([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73])

In [15]:
# calculate the trend values of each segment
ICV_trend = {}
for key in ICV_segments.keys():
    print(key)
    ICV_trend[key] = [data_process.mk_test(x)[0]*10.0 for x in ICV_segments[key]]

10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73


In [16]:
# # Assuming ICV_segments is a dictionary with segment_length as keys and list of DataArray segments as values
# max_num_segments = max(len(segments) for segments in ICV_segments.values())
# segment_lengths = ICV_segments.keys()

# # Create a new Dataset to hold the new arrays
# new_ds = xr.Dataset()

# for segment_length in segment_lengths:
#     segments_list = ICV_segments[segment_length]
#     # print(segments_list)
    
#     # Pad the segments list to have the same number of segments
#     padded_segments = segments_list.copy()
#     while len(padded_segments) < max_num_segments:
#         # Create a DataArray filled with NaNs to match the shape of the segments
#         nan_segment = xr.full_like(padded_segments[0], np.nan)
#         padded_segments.append(nan_segment)
    
#     # Create a coordinate for the new segment dimension
#     segment_coord = range(max_num_segments)
    
#     # Concatenate the padded segments with the new segment coordinate
#     concatenated = xr.concat(padded_segments, dim=segment_coord)
    
#     # Assign a specific name to the new dimension
#     concatenated = concatenated.rename({'concat_dim': 'segment'})
    
#     # Add the new DataArray to the new dataset
#     new_ds[f'ICV_segments_{segment_length}yr'] = concatenated

In [17]:
# ds_combined = xr.merge([ds_MME_HadCRUT5_1850_2022, new_ds])

In [18]:
# ds_combined

In [19]:
# check the minimum and maximum of the new variable
# ds_combined['ICV_segments_10yr'].min().values, ds_combined['ICV_segments_10yr'].max().values

In [20]:
ICV_trend

{10: [-0.22944176338146172,
  -0.17700287036753357,
  0.057884761342091265,
  0.06198824431303941,
  0.1373927319890429,
  0.41350881255356303,
  0.48273139123590614,
  0.4618200804296862,
  0.2674224443349296,
  0.26301916635662403,
  0.13477103483410843,
  0.0513637693912071,
  -0.11048731218213953,
  -0.2568317019337232,
  -0.5026424743624512,
  -0.4383743909687656,
  -0.2445218682858617,
  -0.03362216115103371,
  0.12839036628824296,
  0.5779527522521571,
  0.617083155682676,
  0.617083155682676,
  0.4331220892212096,
  0.3014802758598122,
  0.22844115055664724,
  0.1855601391997286,
  0.28804459269321214,
  0.13150248504007006,
  -0.1397201181463527,
  -0.06610868589078193,
  0.1795091020118431,
  -0.12896833794664486,
  -0.3594585686515128,
  -0.6329238512728765,
  -0.5581530851463036,
  -0.7512500532707623,
  -0.4755561600146702,
  -0.26598091586423206,
  -0.21051458101542478,
  0.28678009828023704,
  0.6687997635573266,
  0.7666839610719222,
  0.6687997635573266,
  0.4027833839

In [ ]:
def check_shapes(data_dict):
    for key, value in data_dict.items():
        if isinstance(value, list):
            print(f"{key}: List of {len(value)} elements")
            for i, item in enumerate(value):
                shape = getattr(item, 'shape', 'No shape attribute')
                print(f"  Element {i}: {value}")
        else:
            shape = getattr(value, 'shape', 'No shape attribute')
            print(f"{key}: {value}")

In [ ]:
# check_shapes(ICV_trend)

In [ ]:
len(ICV_trend[20])

In [ ]:
type(ICV_trend)

In [ ]:
print(max(len(segments) for segments in ICV_trend.values()))

### Transform the dictionary to the dataset

In [21]:
# Assuming ICV_segments is a dictionary with segment_length as keys and list of segments as values
max_num_segments = max(len(segments) for segments in ICV_trend.values())
segment_lengths = ICV_trend.keys()

# Create a new Dataset to hold the new arrays
new_ds = xr.Dataset()

for segment_length in segment_lengths:
    trend_list = ICV_trend[segment_length]
    # print(trend_list)
    
    # Pad the trend list to have the same number of trend
    padded_trend = trend_list.copy()
    print(type(padded_trend))
    # transform the list to DataArray
    padded_trend = [xr.DataArray(x) for i, x in enumerate(padded_trend)]
    while len(padded_trend) < max_num_segments:
        # Create a DataArray filled with NaNs to match the shape of the trend
        nan_segment = xr.full_like(padded_trend[0], np.nan)
        padded_trend.append(nan_segment)
    
    # Create a coordinate for the new segment dimension
    segment_coord = range(max_num_segments)
    
    # Concatenate the padded trend with the new segment coordinate
    concatenated = xr.concat(padded_trend, dim=segment_coord)
    
    # Assign a specific name to the new dimension
    concatenated = concatenated.rename({'concat_dim': 'segment'})
    
    # Add the new DataArray to the new dataset
    new_ds[f'ICV_trend_{segment_length}yr'] = concatenated

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>


In [22]:
new_ds

<xarray.Dataset>
Dimensions:         (segment: 164)
Coordinates:
  * segment         (segment) int64 0 1 2 3 4 5 6 ... 158 159 160 161 162 163
Data variables: (12/64)
    ICV_trend_10yr  (segment) float64 -0.2294 -0.177 0.05788 ... -0.07076 -0.279
    ICV_trend_11yr  (segment) float64 -0.1751 -0.07889 -0.05228 ... -0.07788 nan
    ICV_trend_12yr  (segment) float64 -0.1014 -0.187 0.08445 ... nan nan
    ICV_trend_13yr  (segment) float64 -0.1754 -0.05826 0.1753 ... nan nan nan
    ICV_trend_14yr  (segment) float64 -0.0726 0.05788 0.3013 ... nan nan nan
    ICV_trend_15yr  (segment) float64 -0.01636 0.17 0.263 0.3079 ... nan nan nan
    ...              ...
    ICV_trend_68yr  (segment) float64 -0.02102 -0.02604 -0.02082 ... nan nan nan
    ICV_trend_69yr  (segment) float64 -0.02568 -0.02375 -0.0262 ... nan nan nan
    ICV_trend_70yr  (segment) float64 -0.02344 -0.0292 -0.03043 ... nan nan nan
    ICV_trend_71yr  (segment) float64 -0.02828 -0.03391 -0.03374 ... nan nan nan
    ICV_trend_72yr  (segment) float64 -0.03295 -0.03493 -0.03802 ... nan nan nan
    ICV_trend_73yr  (segment) float64 -0.03452 -0.03974 -0.0399 ... nan nan nan

In [23]:
# extract the trend pattern of each interval of segments
"""
drop the nan values in the trend pattern of each interval of segments
"""
# segements_10yr_trend = ds_combined['ICV_segments_10yr_trend'].dropna(dim='segment')
# segements_11yr_trend = ds_combined['ICV_segments_11yr_trend'].dropna(dim='segment')
# segements_12yr_trend = ds_combined['ICV_segments_12yr_trend'].dropna(dim='segment')
# segements_13yr_trend = ds_combined['ICV_segments_13yr_trend'].dropna(dim='segment')
# segements_14yr_trend = ds_combined['ICV_segments_14yr_trend'].dropna(dim='segment')
# segements_15yr_trend = ds_combined['ICV_segments_15yr_trend'].dropna(dim='segment')

'\ndrop the nan values in the trend pattern of each interval of segments\n'

In [24]:
# segements_15yr_trend

In [25]:
# save the trend pattern of each segment
dir_out = '/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/Figure4/data/'
new_ds.to_netcdf(dir_out + 'Internal_SOP_mean_trend_segmented.nc')